In [16]:
import torch.nn as nn
from diffusers import StableDiffusionPipeline
from transformers import CLIPModel, CLIPTokenizer
import torch
import torch.nn.init as init
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from torchvision import transforms
import loralib as lora
from torch import autocast
model_id = "C:/Users/19793/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/1d0c4ebf6ff58a5caecab40fa1406526bca4b5b9"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to('cuda')
output_dim = 128
input_dim = 128
# def dummy(images, **kwargs):
#     return (images, False)
# pipe.safety_checker = dummy

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [17]:
class LoRALayer(nn.Module):
    def __init__(self, input_dim, output_dim, rank):
        super(LoRALayer, self).__init__()
        # 初始化LoRA参数
        self.lora_A = nn.Parameter(torch.randn(input_dim, rank))
        self.lora_B = nn.Parameter(torch.zeros(rank, output_dim))

        init.xavier_uniform_(self.lora_A)
        init.xavier_uniform_(self.lora_B)
    def forward(self, x):
        # 应用LoRA适应
        delta_weight = self.lora_A @ self.lora_B
        x = x.squeeze(1)
        x = x.to(dtype=torch.float)
        return x @ delta_weight

#replace the output layer of the stable diffusion pipeline with lora
class LoRADiffusionPipeline(nn.Module):
    def __init__(self, rank=5, scale=20, *args, **kwargs):
        super(LoRADiffusionPipeline,self).__init__()
        self.lora = LoRALayer(input_dim, output_dim, rank)
        self.scale = scale

    def forward(self, x):
        with autocast("cuda"):
            y1 = pipe(x,guidance_scale=self.scale).images
        y1 = torch.stack([transform(image) for image in y1])
        y2 = self.lora.forward(y1)
        return y2

transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

class emojiDataset(Dataset):
    def __init__(self, csv_file, root_dir):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
    
    def __len__(self):
        return len(self.data_frame)
    
    def __getitem__(self, idx):
        text = self.data_frame.iloc[idx, 0]
        image_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 1])
        image = Image.open(image_name)
        image = transform(image)
        # #change image to 1-dimensional
        # image = image.view(3, -1)
        return text,image

In [18]:
modified_model = LoRADiffusionPipeline()
lora.mark_only_lora_as_trainable(modified_model)
# Define your loss function
criterion = nn.MSELoss()
# tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32") 
# Create an optimizer
optimizer = torch.optim.Adam(modified_model.lora.parameters(), lr=0.01)

checkpoint_dir = 'model/direct_lora/checkpoints/'
os.makedirs(checkpoint_dir, exist_ok=True)
train_outputs_dir = 'model/direct_lora/train_outputs/'
os.makedirs(train_outputs_dir, exist_ok=True)

dataset=emojiDataset(csv_file='emoji_image/emoji.csv', root_dir='emoji_image/emoji')
dataloader=DataLoader(dataset, batch_size=1, shuffle=True)
# Iterate over your training data
for epoch in range(5):
    loss_epoch = 0
    for idx, batch in enumerate(dataloader):
        # Zero the gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = modified_model(batch[0][0])
        # outputs = outputs.view(3, -1)
        outputs = outputs.to(dtype=torch.float)
        # Calculate the loss
        outputs = outputs.squeeze(0)
        loss = criterion(outputs, batch[1][0])
        # Backward pass and update the weights
        loss.backward()
        optimizer.step()
        # Save model checkpoint
        print(f"Epoch{epoch} {idx+1}/{len(dataset)}, Loss: {loss.item()}")
        loss_epoch += loss.item()
        if idx % 50 == 49:
            torch.save(modified_model.state_dict(), checkpoint_dir + f"model_epoch{epoch}_{idx}.pt")
            #save outputs as images
            torch.save(outputs, train_outputs_dir + f"outputs_epoch{epoch}_{idx}.pt")

        # Print the average loss for this epoch
    print(f"Epoch{epoch}, Average loss: {loss_epoch/len(dataset)}")
torch.save(modified_model.state_dict(), checkpoint_dir + f"model_final.pt")


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch0 1/700, Loss: 0.4431058466434479


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch0 2/700, Loss: 0.3531743288040161


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


Epoch4 205/700, Loss: 0.2920684516429901


  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 